In [ ]:
import pandas as pd
import numpy as np
from binance.client import Client
from datetime import datetime, timedelta, timezone
import os
import pandas_ta as ta
from tqdm import tqdm
import time 
import pytz
import os
# os.system('cls' if os.name == 'nt' else 'clear')
from IPython.display import clear_output
# clear_output(wait=True)

import importlib
import BaseFunctions
importlib.reload(BaseFunctions)
from BaseFunctions import *

import StrategyList
importlib.reload(StrategyList)
from StrategyList import *

import VariableCreation
importlib.reload(VariableCreation)
from VariableCreation import *

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load API credentials from environment variables
API_KEY = os.getenv("BinanceAPI_250502")
API_SECRET = os.getenv("BinanceSecret_250502")

import warnings
warnings.filterwarnings("ignore")

client = Client(API_KEY, API_SECRET) if API_KEY and API_SECRET else Client()

BackTime = "2024-04-01 00:00:00"
# Convert string to naive datetime object
BackTime = datetime.strptime(BackTime, '%Y-%m-%d %H:%M:%S')
# Make it timezone-aware (UTC)
BackTime = BackTime.replace(tzinfo=timezone.utc)
# print(BackTime)
# BackTime = 1735689600000

BackTime = int(BackTime.timestamp() * 1000)  # Convert to milliseconds
Interval = '5m'
current_time = time.time()
pair = 'BTCUSDT'

BuySellFlag = 'Hold'
BuyCounter = 0
SellCounter = 0

from datetime import datetime; import pytz; epoch_to_utc = lambda epoch: datetime.fromtimestamp(epoch / 1000.0 if epoch > 1e10 else epoch, tz=pytz.UTC).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
current_time = int(time.time() * 1000)  # Current time in ms
interval_ms = 5 * 60 * 1000  # 5 minutes in ms

# 2. Chunking logic for END-TIME based requests
time_chunks = []
current_end = BackTime + (1000 * interval_ms)  # First chunk ends after 1000 candles

while current_end <= current_time:
    time_chunks.append(current_end)
    current_end += 1000 * interval_ms  # Move window forward
    
from concurrent.futures import ThreadPoolExecutor, as_completed
df2 = pd.DataFrame()
with ThreadPoolExecutor(max_workers=5) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(get_candles_data, pair, Interval, 1000, end, client) for end in time_chunks]
    
    # Process results as they complete
    for future in as_completed(futures):
        df1 = future.result()
        if not df1.empty:
            df2 = pd.concat([df2, df1], ignore_index=True)

In [ ]:
time_counts = df2['Open Time'].value_counts()
print(time_counts[time_counts > 1])

print(df2['Open Time'].min())

print(df2['Open Time'].max())

print(df2.shape)

In [ ]:
import multiprocessing
cores_mp = multiprocessing.cpu_count()

def process_time(current_time, df2, pair, client):
    """Processes a single timestamp and returns the result."""
    try:
        # Get only NEW data since last iteration
        new_data = df2[df2['epochTime'] == current_time]
        window_data = pd.concat([df2[df2['epochTime'] < current_time], new_data])
        # Clear the console at the start of each iteration
        clear_output(wait=True)
        
        # Process only when we have sufficient history
        if len(window_data) > 100:  # Minimum window size
            processed = create_variablesV2(window_data.copy(), pair, client, current_time)
            strategized = allstrategiesv2(processed)
            
            # Return only the latest result to save memory
            if not strategized.empty:
                return strategized.iloc[[-1]]  # Only keep last row
    except Exception as e:
        print(f"Error at {current_time}: {str(e)}")
    return None

def parallel_process(df2, pair, client):
    """Processes all timestamps in parallel."""
    # Convert timestamps once
    df2['Close Time'] = pd.to_datetime(df2['Close Time'])
    df2['epochTime'] = (df2['Close Time'].astype('int64') // 10**6)
    
    # Sort by time and get unique epochs
    df2 = df2.sort_values('epochTime')
    unique_times = df2['epochTime'].unique()
    
    results = []
    
    with ThreadPoolExecutor(max_workers=cores_mp) as executor:  # Limit to 10 threads
        # Submit tasks to the executor
        futures = [executor.submit(process_time, current_time, df2, pair, client) for current_time in unique_times]
        
        # Process results as they complete
        for future in tqdm(as_completed(futures), total=len(unique_times), desc="Processing"):
            result = future.result()
            if result is not None:
                results.append(result)
                
                # Optional: Clear memory periodically
                if len(results) % 1000 == 0:
                    pd.concat(results).to_parquet(f"partial_{len(results)}.parquet")
                    results = []
                
    
    # Final concatenation
    final_df = pd.concat(results, ignore_index=True)
    return final_df

# Example usage
final_df = parallel_process(df2, pair, client)

In [ ]:
final_df.to_csv('BTCUSDT2024Onwards_AllVarsAndStats.csv', index=False)

In [ ]:
# Convert timestamps once
df2['Close Time'] = pd.to_datetime(df2['Close Time'])
df2['epochTime'] = (df2['Close Time'].astype('int64') // 10**6)

# Sort by time and get unique epochs
df2 = df2.sort_values('epochTime')
unique_times = df2['epochTime'].unique()

# Pre-allocate results list instead of growing DataFrame
results = []
window_data = pd.DataFrame()

for current_time in tqdm(unique_times, desc="Processing"):
    try:
        # Get only NEW data since last iteration (much more efficient)
        new_data = df2[df2['epochTime'] == current_time]
        window_data = pd.concat([window_data, new_data])
        
        # Process only when we have sufficient history
        if len(window_data) > 100:  # Minimum window size
            processed = create_variablesV2(window_data.copy(), pair, client, current_time)
            strategized = allstrategiesv2(processed)
            
            # Append only the latest result to save memory
            if not strategized.empty:
                results.append(strategized.iloc[[-1]])  # Only keep last row
                
        # Optional: Clear memory periodically
        if len(results) % 1000 == 0:
            pd.concat(results).to_parquet(f"partial_{current_time}.parquet")
            results = []
            
    except Exception as e:
        print(f"Error at {current_time}: {str(e)}")
        continue

# Final concatenation
final_df = pd.concat(results, ignore_index=True)